https://www.analyticsvidhya.com/blog/2024/06/rag-pipelines-with-llamaindex-and-trulens/

llamaindex+trulens

初步结论：

- 能跑通
- 虽然全部使用openai api了，但是依然 `groundedness_measure_with_cot_reasons None`

## 准备

In [1]:
%%time
%%capture

!pip install trulens-eval==0.33.0

CPU times: user 7.46 ms, sys: 12.4 ms, total: 19.9 ms
Wall time: 2 s


In [2]:
%%time

import os

base_url = "http://ape:3000/v1"
api_key = "sk-bJP6QSnUfjAYeYeE505d3eBf63A643BeB0B8E350Df9b7750"

os.environ["OPENAI_API_KEY"] = api_key
os.environ["OPENAI_API_BASE"] = base_url

CPU times: user 15 µs, sys: 3 µs, total: 18 µs
Wall time: 19.6 µs


## 创建 RAG

In [3]:
%%time

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

# Load documents from the data directory
documents = SimpleDirectoryReader("data2").load_data()

CPU times: user 2.61 s, sys: 384 ms, total: 2.99 s
Wall time: 2.61 s


In [4]:
%%time

index = VectorStoreIndex.from_documents(documents)

CPU times: user 1.21 s, sys: 24.5 ms, total: 1.23 s
Wall time: 4.97 s


In [5]:
%%time

query_engine = index.as_query_engine()

CPU times: user 34.7 ms, sys: 3.85 ms, total: 38.5 ms
Wall time: 38.1 ms


In [6]:
%%time

response = query_engine.query("What did the author do growing up?")
print(response)

The author worked on writing and programming outside of school before college.
CPU times: user 73.1 ms, sys: 94 µs, total: 73.2 ms
Wall time: 2.27 s


## 使用 TruLens

In [7]:
%%time

from trulens_eval import Tru

tru = Tru()
tru.reset_database()

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.
CPU times: user 2.6 s, sys: 204 ms, total: 2.8 s
Wall time: 2.83 s


In [8]:
%%time

from trulens_eval.feedback.provider import OpenAI

provider = OpenAI()

CPU times: user 82.7 ms, sys: 3.82 ms, total: 86.5 ms
Wall time: 86 ms


In [9]:
%%time

from trulens_eval.app import App
from trulens_eval import Feedback

context = App.select_context(query_engine)

# Define a groundedness feedback function
f_groundedness = (
    Feedback(provider.groundedness_measure_with_cot_reasons)
    .on(context.collect())  # Collect context chunks into a list
    .on_output()
)

✅ In groundedness_measure_with_cot_reasons, input source will be set to __record__.app.query.rets.source_nodes[:].node.text.collect() .
✅ In groundedness_measure_with_cot_reasons, input statement will be set to __record__.main_output or `Select.RecordOutput` .
CPU times: user 9.48 ms, sys: 0 ns, total: 9.48 ms
Wall time: 9.28 ms


In [10]:
# Question/answer relevance between overall question and answer.
f_answer_relevance = (
    Feedback(provider.relevance)
    .on_input_output()
)

✅ In relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .


In [11]:
import numpy as np

# Question/statement relevance between question and each context chunk.
f_context_relevance = (
    Feedback(provider.context_relevance_with_cot_reasons)
    .on_input()
    .on(context)
    .aggregate(np.mean)
)

✅ In context_relevance_with_cot_reasons, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In context_relevance_with_cot_reasons, input context will be set to __record__.app.query.rets.source_nodes[:].node.text .


In [12]:
%%time

from trulens_eval import TruLlama

# Initialize the recorder
tru_query_engine_recorder = TruLlama(query_engine,
    app_id='LlamaIndex_App1',
    feedbacks=[f_groundedness, f_answer_relevance, f_context_relevance])

CPU times: user 370 ms, sys: 4.12 ms, total: 374 ms
Wall time: 382 ms


In [13]:
%%time

import nltk
# [nltk_data] Error loading punkt: <urlopen error [Errno 111] Connection
nltk.set_proxy('http://myproxy:7890')

# Use the recorder as a context manager to log queries
with tru_query_engine_recorder as recording:
    response = query_engine.query("What did the author do growing up?")
    print(response)

The author worked on writing and programming outside of school before college.
CPU times: user 2.11 s, sys: 27.2 ms, total: 2.14 s
Wall time: 6.27 s


In [14]:
%%time

rec = recording.get() 

CPU times: user 6 µs, sys: 0 ns, total: 6 µs
Wall time: 6.68 µs


In [15]:
tru.get_leaderboard(app_ids=[])

,latency,total_cost
app_id,,
LlamaIndex_App1,5.0,0.003127


In [16]:
for feedback, feedback_result in rec.wait_for_feedback_results().items():
    print(feedback.name, feedback_result.result)

groundedness_measure_with_cot_reasons None
relevance None
context_relevance_with_cot_reasons None


In [17]:
records, feedback = tru.get_records_and_feedback(app_ids=["LlamaIndex_App1"])
records.head()

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,ts,context_relevance_with_cot_reasons_calls,groundedness_measure_with_cot_reasons_calls,relevance_calls,latency,total_tokens,total_cost
0,LlamaIndex_App1,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.core.query_en...,record_hash_525e9cc635a9e8fbf43c25085f860f6a,"""What did the author do growing up?""","""The author worked on writing and programming ...",-,"{""record_id"": ""record_hash_525e9cc635a9e8fbf43...","{""n_requests"": 2, ""n_successful_requests"": 3, ...","{""start_time"": ""2024-08-06T16:35:37.545890"", ""...",2024-08-06T16:35:43.331048,[],[],[],5,2096,0.003127
